<a href="https://colab.research.google.com/github/lagypin/lagypin/blob/main/NaNDA_Usage_Metrics_v04.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# 📊 NaNDA Usage Statistics Monthly Scraper
# Includes ICPSR and openICPSR studies

import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
from google.colab import drive
import re
import json
from datetime import datetime

# Mount your Google Drive
drive.mount('/content/drive')

# 🕒 Get current timestamp
scrape_date = datetime.now().strftime('%Y-%m-%d')
print(f"🕒 Starting NaNDA scrape for {scrape_date}")

# 📋 Define known NaNDA study IDs
nanda_ids = [
    "38567", "38649", "38974", "39093", "39378", "38559", "38598", "38579",
    "38586", "38597", "38585", "38605", "38569", "38528", "38580", "38584",
    "38606", "38506", "38858", "110641", "110663", "111107", "111109", "115006", "115323", "115404", "115407", "115408", "115543",
    "115967", "115972", "115973", "115981", "117163", "117866", "117921", "119451", "119803", "120088",
    "120462", "120463", "120907", "121741", "123001", "123042", "123541", "123542", "123801", "123802",
    "124721", "124801", "125223", "125781", "126082", "127042", "127262", "127681", "127682", "128281",
    "128282", "128841", "128862", "130282", "130542", "134561", "141121", "155022", "155025", "156024",
    "156041", "156042", "156043", "156045", "159902", "159941", "159961", "159981", "160261", "160262",
    "190141", "207966", "208207", "208366", "208682", "208684", "208751", "208906", "208907", "209050",
    "209163", "209164", "209313", "209324", "210581", "220701", "222263", "222901", "230941" "237305"
]

base_url = "https://archive.icpsr.umich.edu/nanda/view/studies/"
results = []

# 🕸️ Step 1: Scrape study-level data
print("🔍 Scraping individual study metrics...")
for study_id in nanda_ids:
    url = f"{base_url}{study_id}/study-details"
    print(f"  Scraping study {study_id}...")

    r = requests.get(url)
    soup = BeautifulSoup(r.text, 'html.parser')

    # Extract dataset name
    dataset_name = "Unknown"
    hero_title_div = soup.find("div", class_="hero-title")
    if hero_title_div:
        h1 = hero_title_div.find("h1")
        if h1:
            dataset_name = h1.text.strip()
            prefix = "National Neighborhood Data Archive (NaNDA):"
            if dataset_name.lower().startswith(prefix.lower()):
                dataset_name = dataset_name[len(prefix):].strip()

    # Extract downloads and citations
    downloads = "NA"
    citations = "NA"

    figures = soup.find_all("div", attrs={"data-testid": "study-stat-tracker"})
    for fig in figures:
        value = fig.find("span", class_="display-2").text.strip()
        label = fig.find("figcaption").text.strip().lower()
        if "download" in label:
            downloads = value
        elif "citation" in label:
            citations = value

    results.append({
        "study_id": study_id,
        "dataset_name": dataset_name,
        "downloads": downloads,
        "citations": citations,
        "scrape_date": scrape_date
    })

    time.sleep(1)  # Be polite to the server

print(f"✅ Collected metrics for {len(results)} studies")

# 🆕 Step 2: Scrape publications data
print("\n📚 Scraping NaNDA publications...")
publications = []
publications_base_url = "https://search.icpsr.umich.edu/search/search/nanda/publications"

# Pagination variables
start = 0
rows = 50
has_more_pages = True

while has_more_pages:
    pub_url = f"{publications_base_url}?start={start}&__COMPOUND_SCOPE__=(SERIESQ%3A1920%20OR%20ARCHIVE%3Ananda)%20AND%20PUBLISH_STATUS%3APUBLISHED&sort=TITLE_SORT%20asc&rows={rows}"
    print(f"  📄 Scraping publications page {start//rows + 1}...")

    try:
        pub_response = requests.get(pub_url)
        page_text = pub_response.text

        # Extract JSON data from the page
        json_start = page_text.find('"response":{"docs":[')
        if json_start != -1:
            json_end = page_text.find('}, searchConfig', json_start)
            if json_end != -1:
                json_start_full = page_text.rfind('{"response":', 0, json_start + 50)
                json_text = page_text[json_start_full:json_end+1]

                try:
                    data = json.loads(json_text)
                    docs = data.get('response', {}).get('docs', [])
                    total_found = data.get('response', {}).get('numFound', 0)

                    print(f"    Found {len(docs)} publications on this page (total: {total_found})")

                    for doc in docs:
                        title = doc.get('TITLE', 'Unknown')
                        authors = ', '.join(doc.get('AUTHORS_SPLIT', [])) if doc.get('AUTHORS_SPLIT') else 'Unknown'
                        year = doc.get('YEAR_PUB', 'Unknown')
                        publication_type = doc.get('RIS_TYPE', 'Unknown')
                        journal = doc.get('JOURNAL', doc.get('PUBLISHER', 'Unknown'))
                        doi = doc.get('DOI', '')

                        # Get referenced study IDs
                        referenced_studies = doc.get('STUDY_NO', [])
                        if not isinstance(referenced_studies, list):
                            referenced_studies = [referenced_studies]

                        publications.append({
                            "title": title,
                            "authors": authors,
                            "year": year,
                            "publication_type": publication_type,
                            "journal": journal,
                            "doi": doi,
                            "referenced_studies": ", ".join([str(s) for s in referenced_studies]) if referenced_studies else "Unknown",
                            "scrape_date": scrape_date
                        })

                    # Check if we need more pages
                    current_page_num = start//rows + 1
                    total_pages = (total_found + rows - 1) // rows

                    if current_page_num < total_pages:
                        start += rows
                        time.sleep(2)  # Be polite between requests
                    else:
                        has_more_pages = False

                except json.JSONDecodeError as e:
                    print(f"    ❌ Error parsing JSON: {e}")
                    has_more_pages = False
            else:
                print(f"    ❌ Could not find end of JSON data")
                has_more_pages = False
        else:
            print(f"    ❌ Could not find JSON data in page")
            has_more_pages = False

    except Exception as e:
        print(f"    ❌ Error scraping publications: {e}")
        break

print(f"✅ Collected {len(publications)} total publications")

# 💾 Step 3: Save data to Google Sheets
from google.colab import auth
from googleapiclient.discovery import build
import gspread
from google.auth import default

# Authenticate and set up Google Sheets access
auth.authenticate_user()
creds, _ = default()
gc = gspread.authorize(creds)

# Define paths and sheet IDs
output_base_path = "/content/drive/Shared drives/ISR-ISR-NaNDAU01/Metrics/"
STUDY_METRICS_SHEET_ID = "1Iyp8Fa6XueBx1uWFaly2R0t_gpobLohu3sKyi--Ujlc"  # 🔧 UPDATE THIS
PUBLICATIONS_SHEET_ID = "1iu_nCIBG19m4NI_v7uxNjBmot2nOb9tqaDxfKGXNxx8"    # 🔧 UPDATE THIS

def save_to_google_sheet(df, sheet_id, worksheet_name="Sheet1", scrape_date=None):
    """Save DataFrame to Google Sheet with smart deduplication"""
    try:
        # Open the Google Sheet
        sheet = gc.open_by_key(sheet_id)
        worksheet = sheet.worksheet(worksheet_name)

        # Get existing data
        existing_data = worksheet.get_all_records()

        if existing_data:
            existing_df = pd.DataFrame(existing_data)

            # For study metrics, remove today's entries to avoid duplicates
            if 'scrape_date' in existing_df.columns and scrape_date:
                if 'study_id' in existing_df.columns:  # Study metrics
                    existing_df_filtered = existing_df[existing_df['scrape_date'] != scrape_date]
                    combined_df = pd.concat([existing_df_filtered, df], ignore_index=True)
                    print(f"  📈 Replaced {len(existing_df) - len(existing_df_filtered)} entries for {scrape_date}")
                else:  # Publications - check for duplicates by title
                    new_pubs = df[~df['title'].isin(existing_df['title'])]
                    if len(new_pubs) > 0:
                        combined_df = pd.concat([existing_df, new_pubs], ignore_index=True)
                        print(f"  📈 Added {len(new_pubs)} new publications")
                    else:
                        combined_df = existing_df
                        print(f"  ℹ️  No new publications found")
            else:
                combined_df = pd.concat([existing_df, df], ignore_index=True)
        else:
            combined_df = df
            print(f"  ✅ Created new sheet with {len(df)} entries")

        # Clear the sheet and update with new data
        worksheet.clear()
        worksheet.update([combined_df.columns.values.tolist()] + combined_df.values.tolist())

        return len(combined_df)

    except Exception as e:
        print(f"  ❌ Error saving to Google Sheet: {e}")
        print(f"  💡 Make sure the sheet ID is correct and you have edit permissions")
        return None

# Save study metrics to Google Sheets
if results:
    studies_df = pd.DataFrame(results)

    if STUDY_METRICS_SHEET_ID != "YOUR_STUDY_METRICS_SHEET_ID":
        print("📊 Saving study metrics to Google Sheets...")
        total_rows = save_to_google_sheet(studies_df, STUDY_METRICS_SHEET_ID, "Sheet1", scrape_date)
        if total_rows:
            print(f"✅ Study metrics saved to Google Sheets ({total_rows} total rows)")

        # Also create a backup CSV file
        backup_path = f"{output_base_path}nanda_usage_stats_backup.csv"
        studies_df.to_csv(backup_path, index=False)
        print(f"💾 Backup CSV saved: {backup_path}")
    else:
        print("⚠️  Please update STUDY_METRICS_SHEET_ID in the script")
        # Fallback to CSV
        studies_path = f"{output_base_path}nanda_usage_stats.csv"
        studies_df.to_csv(studies_path, index=False)
        print(f"💾 Saved to CSV instead: {studies_path}")

# Save publications to Google Sheets
if publications:
    publications_df = pd.DataFrame(publications)

    if PUBLICATIONS_SHEET_ID != "YOUR_PUBLICATIONS_SHEET_ID":
        print("📚 Saving publications to Google Sheets...")
        total_rows = save_to_google_sheet(publications_df, PUBLICATIONS_SHEET_ID, "Sheet1", scrape_date)
        if total_rows:
            print(f"✅ Publications saved to Google Sheets ({total_rows} total rows)")

        # Also create a backup CSV file
        backup_path = f"{output_base_path}nanda_publications_backup.csv"
        publications_df.to_csv(backup_path, index=False)
        print(f"💾 Backup CSV saved: {backup_path}")
    else:
        print("⚠️  Please update PUBLICATIONS_SHEET_ID in the script")
        # Fallback to CSV
        pubs_path = f"{output_base_path}nanda_publications.csv"
        publications_df.to_csv(pubs_path, index=False)
        print(f"💾 Saved to CSV instead: {pubs_path}")

# 📊 Display summary
print(f"\n🎯 Scrape completed for {scrape_date}!")
print(f"  📊 Study metrics: {len(results)} studies")
print(f"  📚 Publications: {len(publications)} total")

# 📈 Show recent data preview
if results:
    print(f"\n📊 Latest study metrics:")
    display_df = pd.DataFrame(results)[['study_id', 'dataset_name', 'downloads', 'citations']]
    print(display_df.to_string(index=False))

# 🔍 Show changes if this isn't the first run
if results:
    try:
        all_studies = pd.read_csv(f"{output_base_path}nanda_usage_stats.csv")
        dates = sorted(all_studies['scrape_date'].unique())

        if len(dates) > 1:
            print(f"\n📈 Changes since last scrape ({dates[-2]}):")
            current_data = all_studies[all_studies['scrape_date'] == dates[-1]]
            previous_data = all_studies[all_studies['scrape_date'] == dates[-2]]

            changes_found = False
            for _, row in current_data.iterrows():
                study_id = row['study_id']
                prev_row = previous_data[previous_data['study_id'] == study_id]

                if not prev_row.empty:
                    prev_downloads = prev_row.iloc[0]['downloads']
                    prev_citations = prev_row.iloc[0]['citations']

                    # Check for download changes
                    if str(row['downloads']).isdigit() and str(prev_downloads).isdigit():
                        download_change = int(row['downloads']) - int(prev_downloads)
                        if download_change > 0:
                            print(f"  📈 Study {study_id}: +{download_change} downloads")
                            changes_found = True

                    # Check for citation changes
                    if str(row['citations']).isdigit() and str(prev_citations).isdigit():
                        citation_change = int(row['citations']) - int(prev_citations)
                        if citation_change > 0:
                            print(f"  📚 Study {study_id}: +{citation_change} citations")
                            changes_found = True

            if not changes_found:
                print("  No changes in downloads or citations since last scrape")

    except Exception as e:
        print("  (Could not calculate changes)")

print(f"\n📁 Data saved to: {output_base_path}")
print("🎉 Monthly scrape complete!")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
🕒 Starting NaNDA scrape for 2025-08-15
🔍 Scraping individual study metrics...
  Scraping study 38567...
  Scraping study 38649...
  Scraping study 38974...
  Scraping study 39093...
  Scraping study 39378...
  Scraping study 38559...
  Scraping study 38598...
  Scraping study 38579...
  Scraping study 38586...
  Scraping study 38597...
  Scraping study 38585...
  Scraping study 38605...
  Scraping study 38569...
  Scraping study 38528...
  Scraping study 38580...
  Scraping study 38584...
  Scraping study 38606...
  Scraping study 38506...
  Scraping study 38858...
  Scraping study 110641...
  Scraping study 110663...
  Scraping study 111107...
  Scraping study 111109...
  Scraping study 115006...
  Scraping study 115323...
  Scraping study 115404...
  Scraping study 115407...
  Scraping study 115408...
  Scraping study 115543...
  Scraping study 115967...
  